In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain.vectorstores import Chroma
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import pickle

Document loader

In [95]:
import glob

pdf_files = glob.glob("src/*.pdf")
pages = []

for pdf_file in pdf_files:
    pages.extend(PyPDFLoader(pdf_file).load_and_split())

for page in pages:
    page.page_content = page.page_content.replace("Imprimerie Officielle de la République Tunisienne", "")


Splitter


In [96]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
len(chunks)


28127

Vectorstores

In [13]:
import os
os.environ["PINECONE_API_KEY"] = "9721efa4-ff98-4bc5-9b28-93919d1657a5"
embedder = HuggingFaceEmbeddings(
    model_name = "EIStakovskii/LaBSE-fr-de"
)

index_name = "pfa"

#docsearch = PineconeVectorStore.from_documents(chunks, embedder, index_name=index_name)

No sentence-transformers model found with name C:\Users\mehdi/.cache\torch\sentence_transformers\EIStakovskii_LaBSE-fr-de. Creating a new one with MEAN pooling.


# Retrieval
## check retriever

In [17]:

query="que'est ce que  l'identifiant unique citoyen ?"
docsearch = PineconeVectorStore(index_name=index_name,embedding=embedder)
#méthode 1 avec retriever
# retriever = docsearch.as_retriever(search_type="mmr")
# matched_docs = retriever.get_relevant_documents(query)
# for i, d in enumerate(matched_docs):
#     print(f"\n## Document {i}\n")
#     print(d.page_content)

#méthode 2 avec mmr
found_docs = docsearch.max_marginal_relevance_search(query, k=5, fetch_k=100)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n\n")

#méthode 3 avec similarity search
# sim_docs = docsearch.similarity_search_with_score(query, k=3)  
# for result in sim_docs:
#     print("\n")
#     print(result[1])
#     print(result[0].page_content)

1. données à caractère personnel. 
Chapitre IV 
Du Conseil de suivi d’utilisation de l'identifiant unique citoyen 


2. Article 3.- L’identifiant unique citoyen est attribué à toute 
personne physique : 
- ayant la nationalité tunisienne, inscrite sur le registre des 
naissances de l’état civil, 
- de nationalité tunisienne née dans un pays étranger, inscrite 
auprès de la mission diplomatique ou consulaire accréditée dans ce 
pays, 
- ayant acquis la nationalité tunisienne par voie de naturalisation. 


3. Il est également interdit d'attribuer le même identifiant à plusieurs 
personnes ou d'attribuer à une personne plusieurs identifiants. 


4. précisant l'origine des droits, l'identité du déclarant, les limites de l'immeuble et les charges le grevant. En cas de copropriété, la déclaration peut émaner d'un seul copropriétaire. 


5. Article 8.-  Le contenu de l’identifiant unique citoyen ainsi que ses 
spécifications techniques et les règles régissant la tenue et la gestion 




In [21]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



new_query= "contexte :  "+ format_docs(found_docs)+ "question : "+ query +" on parle toujours en tunisie et répondez en français"



In [30]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt = prompt.invoke({"context":format_docs(found_docs), "question":query})
print(prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: que'est ce que  l'identifiant unique citoyen ? \nContext: données à caractère personnel. \nChapitre IV \nDu Conseil de suivi d’utilisation de l'identifiant unique citoyen\n\nArticle 3.- L’identifiant unique citoyen est attribué à toute \npersonne physique : \n- ayant la nationalité tunisienne, inscrite sur le registre des \nnaissances de l’état civil, \n- de nationalité tunisienne née dans un pays étranger, inscrite \nauprès de la mission diplomatique ou consulaire accréditée dans ce \npays, \n- ayant acquis la nationalité tunisienne par voie de naturalisation.\n\nIl est également interdit d'attribuer le même identifiant à plusieurs \npersonnes ou d'attribuer à une personne plusieurs identifiants.\n\nprécisant

In [31]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm.invoke(prompt)